In [ ]:
import erddaplogs.plot_functions as plot_functions
import matplotlib.pyplot as plt
import polars as pl
import numpy as np

In [ ]:
req_files = pl.read_csv("https://erddap.observations.voiceoftheocean.org/erddap/files/requests/.csv")
df = pl.read_csv(f"https://erddap.observations.voiceoftheocean.org/erddap/files/requests/{req_files['Name'].to_list()[0]}",try_parse_dates=True)

In [ ]:
#df = pl.read_csv("https://erddap.observations.voiceoftheocean.org/erddap/tabledap/requests.csvp",try_parse_dates=True, infer_schema_length=0)
#df = df.rename({'time (UTC)': 'datetime'})
df

In [ ]:
ax  = plot_functions.plot_bytes(df, days=1)
ax.set(ylim=(0, 10))

In [ ]:
plot_functions.plot_most_popular(df, "erddap_request_type");

In [ ]:
plot_functions.plot_most_popular(df, "file_type");

In [ ]:
plot_functions.plot_most_popular(df, "dataset_type");

### Dig into http status codes

succes rate

In [ ]:
print(f"Proportion of successful status codes (http 200): {np.round(100 * df.filter(pl.col('status_code')==200).shape[0]/df.shape[0], 1)} %")

In [ ]:
plot_functions.plot_most_popular(df, "status_code");

what responses were redirected?

In [ ]:
df.filter(pl.col("status_code")==301)["url"].to_list()

forbidden requests

In [ ]:
df.filter(pl.col("status_code")==403)["url"].to_list()[:50]

## Locations

In [ ]:
locs = pl.read_csv("https://erddap.observations.voiceoftheocean.org/erddap/tabledap/locations.csvp")
from iso3166 import countries
country_codes = locs['countryCode'].to_list()
countries_list = []
for code in country_codes:
    try:
        countries_list.append(countries.get(code).name)
    except KeyError:
        countries_list.append('unkown')
locs = locs.with_columns(country=np.array(countries_list))
locs


In [ ]:
locs_by_nation = locs.group_by('country').sum().sort('total_requests', descending=True).head(20)
fig, ax = plt.subplots(figsize=(8, 6))
ax.barh(
    np.arange(len(locs_by_nation)),
    locs_by_nation["total_requests"].to_list(),
    tick_label=locs_by_nation["country"].to_list(),
)
ax.set_title(f"ERDDAP request most common request nations")
ax.invert_yaxis()